In [12]:
import pandas as pd
from geopy.distance import vincenty

In [2]:
df = pd.read_csv('train.csv')

In [13]:
df.head(1)

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,1,0


In [15]:
df['distance_1'] = range(len(df.Latitude))

In [19]:
df['distance_1'] = df.distance_1.apply(lambda x: vincenty((df.Latitude[x], df.Longitude[x]),(41.995, -87.933)).miles)

In [21]:
df['distance_2'] = range(len(df.Latitude))
df['distance_2'] = df.distance_2.apply(lambda x: vincenty((df.Latitude[x], df.Longitude[x]),(41.786, -87.752)).miles)

In [27]:
df['airport'] = (df['distance_1'] > df['distance_2'])*1+1

In [30]:
df.head(1)

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent,distance_1,distance_2,airport
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,1,0,7.345852,11.913475,1


In [5]:
df2 = pd.read_csv('spray.csv')

In [6]:
df2.head()

,Date,Time,Latitude,Longitude
0,2011-08-29,6:56:58 PM,42.391623,-88.089163
1,2011-08-29,6:57:08 PM,42.391348,-88.089163
2,2011-08-29,6:57:18 PM,42.391022,-88.089157
3,2011-08-29,6:57:28 PM,42.390637,-88.089158
4,2011-08-29,6:57:38 PM,42.390410,-88.088858


In [7]:
df3 = pd.read_csv('weather.csv')

In [9]:
df3.shape

(2944, 22)

In [11]:
df3.head()

,Station,Date,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,...,CodeSum,Depth,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
0,1,2007-05-01,83,50,67,14,51,56,0,2,...,,0,M,0.0,0.00,29.10,29.82,1.7,27,9.2
1,2,2007-05-01,84,52,68,M,51,57,0,3,...,,M,M,M,0.00,29.18,29.82,2.7,25,9.6
2,1,2007-05-02,59,42,51,-3,42,47,14,0,...,BR,0,M,0.0,0.00,29.38,30.09,13.0,4,13.4
3,2,2007-05-02,60,43,52,M,42,47,13,0,...,BR HZ,M,M,M,0.00,29.44,30.08,13.3,2,13.4
4,1,2007-05-03,66,46,56,2,40,48,9,0,...,,0,M,0.0,0.00,29.39,30.12,11.7,7,11.9


In [31]:
df_merged = pd.merge(df, df3, left_on = ['Date','airport'], right_on = ['Date','Station'], how= 'left')

In [32]:
df_merged.shape

(10506, 36)

In [34]:
df_merged.head(2)

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,...,CodeSum,Depth,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,...,BR HZ,0,M,0.0,0.00,29.39,30.11,5.8,18,6.5
1,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,...,BR HZ,0,M,0.0,0.00,29.39,30.11,5.8,18,6.5


In [36]:
pd.set_option('max_columns',len(df_merged.columns))

In [37]:
df_merged.head(1)

,Date,Address,Species,Block,Street,Trap,AddressNumberAndStreet,Latitude,Longitude,AddressAccuracy,NumMosquitos,WnvPresent,distance_1,distance_2,airport,Station,Tmax,Tmin,Tavg,Depart,DewPoint,WetBulb,Heat,Cool,Sunrise,Sunset,CodeSum,Depth,Water1,SnowFall,PrecipTotal,StnPressure,SeaLevel,ResultSpeed,ResultDir,AvgSpeed
0,2007-05-29,"4100 North Oak Park Avenue, Chicago, IL 60634,...",CULEX PIPIENS/RESTUANS,41,N OAK PARK AVE,T002,"4100 N OAK PARK AVE, Chicago, IL",41.95469,-87.800991,9,1,0,7.345852,11.913475,1,1,88,60,74,10,58,65,0,9,0421,1917,BR HZ,0,M,0.0,0.00,29.39,30.11,5.8,18,6.5


In [41]:
df_merged.AddressAccuracy.value_counts()

8    4628
9    3980
5    1807
3      91
Name: AddressAccuracy, dtype: int64

In [44]:
df_merged.drop(['Address','Block','Street','AddressNumberAndStreet','AddressAccuracy','distance_1','distance_2','airport','Station'], axis = 1, inplace = True)

In [45]:
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10506 entries, 0 to 10505
Data columns (total 27 columns):
Date            10506 non-null object
Species         10506 non-null object
Trap            10506 non-null object
Latitude        10506 non-null float64
Longitude       10506 non-null float64
NumMosquitos    10506 non-null int64
WnvPresent      10506 non-null int64
Tmax            10506 non-null int64
Tmin            10506 non-null int64
Tavg            10506 non-null object
Depart          10506 non-null object
DewPoint        10506 non-null int64
WetBulb         10506 non-null object
Heat            10506 non-null object
Cool            10506 non-null object
Sunrise         10506 non-null object
Sunset          10506 non-null object
CodeSum         10506 non-null object
Depth           10506 non-null object
Water1          10506 non-null object
SnowFall        10506 non-null object
PrecipTotal     10506 non-null object
StnPressure     10506 non-null object
SeaLevel        1050